In [1]:
import pandas as pd
import numpy as np

import plotly.express as px
from folium import Map
from folium.plugins import HeatMap
from IPython.display import Image

In [2]:
data = pd.read_csv('/home/starczyn/Solar_Project/data/TTS_data.csv')

/tmp/ipykernel_531/3678986517.py:1: DtypeWarning: Columns (2,3,13,28,29,30,31,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/home/starczyn/Solar_Project/data/TTS_data.csv')


In [3]:
data.keys()

Index(['Unnamed: 0', 'dataProvider1', 'dataProvider2', 'program1ProjectID',
       'PTODate_orProxy_', 'systemSizeInDCSTC_KW_', 'totalInstalledCost___',
       'Up_FrontCashIncentive___', 'customerSegment', 'is_expansion',
       'is_multiphase', 'tracking_Y_N_', 'ground_mounted_Y_N_',
       'hostCustomerZip__4_', 'hostCustomerCity', 'state',
       'city_state_country', 'latitude', 'longitude',
       'utilityServiceTerritory', 'x3rdPartyOwned_1_0_', 'installer_clean',
       'self_install_Y_N_', 'azimuth_1', 'tilt_1', 'mod_manuf1_clean',
       'mod_model1_clean', 'moduleQty_1', 'mod_manuf2_clean',
       'mod_model2_clean', 'mod_manuf3_clean', 'mod_model3_clean',
       'additionalModuleModels_Y_N_', 'mod_tech1', 'mod_tech2', 'mod_tech3',
       'mod_BIPV1', 'mod_bifacial1', 'mod_nameplate_capacity1',
       'mod_efficiency1', 'inv_manuf1_clean', 'inv_model1_clean',
       'inverterQuantity_1', 'additionalInverterModels_Y_N_', 'inv_microinv1',
       'inv_battery_hybrid1', 'inv_bui

In [4]:
data.describe()

,Unnamed: 0,systemSizeInDCSTC_KW_,totalInstalledCost___,Up_FrontCashIncentive___,is_expansion,is_multiphase,tracking_Y_N_,ground_mounted_Y_N_,latitude,longitude,...,mod_efficiency1,inverterQuantity_1,additionalInverterModels_Y_N_,inv_microinv1,inv_battery_hybrid1,inv_builtin_meter1,inv_outputcapacity1,dc_optimizer,ILR,TotalModuleQty
count,617580.00000,617580.000000,6.175800e+05,6.175800e+05,617580.000000,617580.000000,617580.000000,617580.000000,612119.000000,612119.000000,...,617580.000000,617580.000000,617580.000000,617580.000000,617580.000000,617580.000000,617580.000000,617580.000000,617580.000000,617580.000000
mean,308789.50000,17.709712,4.416344e+04,3.847482e+03,0.020061,0.004174,-0.469743,-0.593343,37.709389,-96.228934,...,0.179208,6.098078,-0.431411,0.253305,-0.168615,0.537011,6.582009,0.361407,0.459918,88.452778
std,178280.13396,199.724302,3.874617e+05,5.288272e+04,0.140208,0.064474,0.505051,0.513471,4.421955,20.441102,...,0.021989,26.997170,0.498295,0.502812,0.455385,0.558846,53.905315,0.521168,1.003017,979.907699
min,0.00000,-1.000000,-1.000000e+00,-1.000000e+00,0.000000,0.000000,-1.000000,-1.000000,13.677775,-162.356948,...,0.054687,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,154394.75000,4.875000,1.214400e+04,0.000000e+00,0.000000,0.000000,-1.000000,-1.000000,33.580612,-114.476032,...,0.160714,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.290000,0.000000,-1.000000,13.180236
50%,308789.50000,6.867000,2.343100e+04,0.000000e+00,0.000000,0.000000,0.000000,-1.000000,37.976852,-105.941835,...,0.180380,1.000000,0.000000,0.000000,0.000000,1.000000,4.000000,0.000000,1.056000,35.187879
75%,463184.25000,9.520000,3.525600e+04,1.879000e+03,0.000000,0.000000,0.000000,0.000000,41.666530,-73.617192,...,0.197531,5.000000,0.000000,1.000000,0.000000,1.000000,7.000000,1.000000,1.186047,55.160657
max,617579.00000,35882.400000,1.111111e+08,6.125000e+06,1.000000,1.000000,1.000000,1.000000,65.980232,-0.416505,...,0.227545,10080.000000,1.000000,1.000000,1.000000,1.000000,3650.134000,1.000000,1.500000,125279.188100
